In [ ]:
import numpy as np
import imageio
from softlearning.models.state_estimation import (
    state_estimator_model, get_dumped_pkl_data, get_seed_data)
import tensorflow as tf
import matplotlib.pyplot as plt
import gzip
import os
import pickle
%matplotlib inline

In [ ]:
image_shape = (32, 32, 3)

obs_keys = ('object_position',
            'object_orientation_cos',
            'object_orientation_sin')
model = state_estimator_model(
    domain='DClaw',
    task='TurnFreeValve3ResetFreeSwapGoal-v0',
    obs_keys_to_estimate=obs_keys, 
    input_shape=image_shape,
#     num_hidden_units=256,
    num_hidden_units=512,
    num_hidden_layers=4,
)

model.summary()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='mean_squared_error')

In [ ]:
for layer in model.layers:
    print(layer.name)
    if 'input' in layer.name:
        continue
    layer.summary()

In [ ]:
with gzip.open(os.path.join('/home/justinvyu/dev/softlearning-vice/goal_classifier/free_screw_state_estimator_data_invisible_claw', 'more_data.pkl'), 'rb') as f:
    data = pickle.load(f)

In [ ]:
def train(model, data, save_path, n_epochs=50):
    assert 'pixels' in data and 'states' in data, 'Invalid training data'
    pixels, states = data['pixels'], data['states']
    history = model.fit(
        x=pixels,
        y=states,
        batch_size=128,
        epochs=n_epochs,
        validation_split=0.05
    )
    
    model.save_weights(save_path)
    return history

history = train(model, data, './state_estimator_antialias_larger_network.h5', n_epochs=100)

In [ ]:
# weights_path = './state_estimator_random_data_50_epochs.h5'
# weights_path = './state_estimator_invisible_claw.h5'
# weights_path = './state_estimator_antialias.h5'
weights_path = './state_estimator_antialias_larger_network.h5'
model.load_weights(weights_path)

In [ ]:
# Aliased, random, invisible
training_pools_base_path = '/home/justinvyu/dev/softlearning-vice/goal_classifier/free_screw_state_estimator_data_invisible_claw/data.pkl'
images, labels = get_dumped_pkl_data(training_pools_base_path)

In [ ]:
# Aliased, on policy, invisible
training_pools_base_path = '/root/softlearning-vice/goal_classifier/free_screw_state_estimator_data_invisible_claw_test/data.pkl'
images, labels = get_dumped_pkl_data(training_pools_base_path)

In [ ]:
# Anti-aliased, random, invisible
training_pools_base_path = '/home/justinvyu/dev/softlearning-vice/goal_classifier/free_screw_state_estimator_data_invisible_claw/data.pkl'
images, labels = get_dumped_pkl_data(training_pools_base_path)

In [ ]:
images, labels = data['pixels'], data['states']

In [ ]:
# Show some photos
plt.axis('off')
test_index = 30009
plt.imshow(images[test_index])
print(labels[test_index])

In [ ]:
# Get samples to calculate metrics on
random_indices = np.random.choice(images.shape[0], size=5000)
test_images = images[random_indices]
test_labels = labels[random_indices]
preds = model.predict(test_images)

In [ ]:
pos_errors = []
angle_errors = []

degrees = lambda x: x * 180 / np.pi
def angle_distance(deg1, deg2):
    phi = np.abs(deg1 - deg2) % 360
    distance = 360 - phi if phi > 180 else phi
    return distance

for i, (test_img, label, pred) in enumerate(zip(test_images, test_labels, preds)):
    pos_error_xy = np.abs(label[:2] - pred[:2])
    pos_error = np.linalg.norm(pos_error_xy)
    pos_error = 15 * pos_error # free box is 30 cm, 15 on each side (-1 -> 1 --> -15 -> 15)
    
    true_angle = np.arctan2(label[3], label[2])
    true_angle = degrees(true_angle)
    pred_angle = np.arctan2(pred[3], pred[2])
    pred_angle = degrees(pred_angle)
    
    angle_error = angle_distance(true_angle, pred_angle)

    pos_errors.append(pos_error)
    angle_errors.append(angle_error)

#     print('\n========== IMAGE #', i, '=========')
#     print('POS ERROR (cm):', pos_error, 'true xy: {}'.format(label[:2]), 'pred xy: {}'.format(pred[:2]))
#     print('ANGLE ERROR (degrees):', angle_error, 'true angle: {}'.format(true_angle), 'pred angle: {}'.format(pred_angle))
#     imageio.imwrite(f'/root/imgs/test{i}.jpg', test_img)

mean_pos_error = np.mean(pos_errors)
mean_angle_error = np.mean(angle_errors)
print('MEAN POS ERROR (CM):', mean_pos_error)
print('MEAN ANGLE ERROR (degrees):', mean_angle_error)

In [ ]:
def display_top_errors(errors, label_str=""):
    errors = np.array(errors)
    ind = np.argpartition(errors, -20)[-20:]
    ind = ind[np.argsort(errors[ind])]
    ind = np.flip(ind) # Order descending
    top_errors = errors[ind]
    
    top_error_imgs, top_error_labels, top_error_preds = test_images[ind], test_labels[ind], preds[ind]
    for i, (error, img, label, pred) in enumerate(zip(top_errors,
                                                      top_error_imgs,
                                                      top_error_labels,
                                                      top_error_preds)):
        print('\n========== IMAGE #', i, '=========')
        plt.axis('off')
        plt.imshow(img)
        print('{} ERROR: {}\n\ntrue: {}\npred: {}'.format(label_str, error, label, pred))
        plt.show()
    
display_top_errors(pos_errors, label_str="POS (cm)")

In [ ]:
display_top_errors(angle_errors, label_str="ANGLE (degrees)")

In [ ]:
first = np.arctan2(-0.23606753, -0.97173665)
second = np.arctan2(0.17438738, -0.9635754)
rad_to_deg = lambda x: 180 * x / np.pi
rad_to_deg(first), rad_to_deg(second)

In [ ]:
def plot_histograms(pos_errors, angle_errors):
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title('Position Errors (cm)')
    plt.hist(pos_errors, bins=30)
    plt.subplot(1, 2, 2)
    plt.title('Angle Errors (deg)')
    plt.hist(angle_errors, bins=30)
    plt.show()
    
plot_histograms(pos_errors, angle_errors)

In [ ]:
def plot_pos_support():
    plt.figure(figsize=(5, 5))
    plt.scatter(test_labels[:, 0], test_labels[:, 1], alpha=0.1, s=5)
    plt.show()
    
plot_pos_support()

def plot_angle_support():
    plt.figure(figsize=(5,5))
    angles = np.arctan2(test_labels[:, 3], test_labels[:, 2])
    plt.hist(angles, bins=50)
    plt.show()
    
plot_angle_support()

In [ ]:
# for label, pred in zip(test_labels, preds):
#     labels_x, labels_y = test_labels[:, 0], test_labels[:, 1]
#     preds_x, preds_y = preds[:, 0], preds[:, 1]
#     print(preds_x.shape, preds_y.shape)
#     dxs, dys = preds_x - labels_x, preds_y - labels_y
#     print(dxs.shape, dys.shape)

import seaborn as sns
sns.set()
def get_noise(size, loc=0, scale=0.02):
    return np.random.normal(loc=loc, scale=scale, size=size)

labels_x, labels_y = test_labels[:, 0], test_labels[:, 1]

# noisy_x, noisy_y = (
#     test_labels[:, 0] + get_noise(test_labels[:, 0].shape),
#     test_labels[:, 1] + get_noise(test_labels[:, 1].shape),
# )
# dxs_noise, dys_noise = noisy_x - labels_x, noisy_y - labels_y
# plt.figure(figsize=(20, 20))
# plt.quiver(labels_x, labels_y, dxs_noise, dys_noise, angles='xy', scale_units='xy', scale=1, width=0.001, alpha=0.5)

preds_x, preds_y = preds[:, 0], preds[:, 1]
dxs, dys = preds_x - labels_x, preds_y - labels_y
plt.figure(figsize=(20, 20))
plt.title('State estimator errors (xy)')

plt.scatter(labels_x, labels_y, c='blue', s=2, label='labels (blue)')
plt.scatter(preds_x, preds_y, c='green', s=2, label='preds (green)')
plt.legend()
plt.quiver(labels_x, labels_y, dxs, dys, angles='xy', scale_units='xy', scale=1, width=0.001, alpha=0.6)

plt.figure(figsize=(10, 10))
plt.title('Position errors')
plt.xlabel('label')
plt.ylabel('predicted')

plt.scatter(labels_x, preds_x, s=0.5, alpha=0.2, label='x')
plt.scatter(labels_y, preds_y, s=0.5, alpha=0.2, label='y')
plt.legend()


In [ ]:
labels_z_cos, labels_z_sin = test_labels[:, 2], test_labels[:, 3]
preds_z_cos, preds_z_sin = preds[:, 2], preds[:, 3]
dzs_cos, dzs_sin = preds_z_cos - labels_z_cos, preds_z_sin - labels_z_sin

# plt.figure(figsize=(20, 20))
# plt.title('State estimator errors (angle)')
# plt.scatter(labels_z_cos, labels_z_sin, c='blue', s=2, label='labels (blue)')
# plt.scatter(preds_z_cos, preds_z_sin, c='green', s=2, label='preds (green)')
# plt.legend()
# # circle = plt.Circle((0, 0), 1, color='black', alpha=.5, fill=False)
# # plt.gcf().gca().add_artist(circle)
# plt.quiver(labels_z_cos, labels_z_sin, dzs_cos, dzs_sin, angles='xy', scale_units='xy', scale=1, width=0.001, alpha=0.6)

labels_angle, preds_angle = (
    np.arctan2(labels_z_sin, labels_z_cos),
    np.arctan2(preds_z_sin, preds_z_cos)
)
print(labels_angle, labels_angle)
plt.figure(figsize=(10, 10))
plt.title('Angle errors')
plt.xlabel('label angle (radians)')
plt.ylabel('predicted angle (radians)')
plt.scatter(labels_angle, preds_angle, s=0.4, alpha=0.25)


In [ ]:
import glob
import os
import pickle
import gzip
import numpy as np
# Aggregate data pools together
path = '/home/justinvyu/dev/softlearning-vice/goal_classifier'

aggregated = {}
for data_dir in glob.iglob(os.path.join(path, 'free_screw_state_estimator*')):
    print(data_dir)
    data_fn = os.path.join(data_dir, 'data.pkl')
    try:
        with gzip.open(data_fn, 'rb') as f:
            try:
                data = pickle.load(f)
                print(data.keys())
            except:
                continue
    except:
        continue
        
    for k, v in data.items():
        print(k, v.shape)
        if k in aggregated:
            aggregated[k] = np.concatenate([
                aggregated.get(k), v])
        else:
            aggregated[k] = v
        print(aggregated[k].shape)
        
aggregated['pixels'].shape

In [ ]:
aggregated['states'].shape
with gzip.open(os.path.join('/home/justinvyu/dev/softlearning-vice/goal_classifier/free_screw_state_estimator_data_invisible_claw', 'more_data.pkl'), 'wb') as f:
    pickle.dump(aggregated, f, protocol=4)